# 欢迎使用：云端Manim·编译器！

### 前往[我的Manim社区](https://manim.notion.site/)


这是一个***临时环境***，你可以在这里编写 Manim，无需在本地安装。具备一些
Python
基础知识会有所帮助！但请记住：**你的更改不会被保存，也无法与他人共享。要保存你的工作，你需要下载笔记本文件**（“文件
\> 下载为 \> 笔记本 (.ipynb)”）。祝你愉快使用！

> *有用的资源：*
>[Manim文档](https://docs.manim.community)

## 设置

我们从导入库中的所有内容开始这个简短的入门指南。运行以下代码单元格来完成导入（聚焦单元格并点击上方的*运行*按钮，或按
`Shift`+`Enter`——你可以在页面顶部的*帮助*菜单中找到更多关于如何导航和使用
Jupyter 笔记本的信息）。

第二行控制在本笔记本中显示***视频的最大宽度***，第三行控制日***志输出的详细程度***。你可以根据自己的喜好调整这两个设置。

In [ ]:
import manim as mn
from manim import *

config.media_width = "75%"
config.verbosity = "WARNING"

print(mn.__version__)

如果你成功执行了该单元格，下方应该会出现一条打印已安装库版本的消息。

## 安装字体

在线版的Manim免费而且方便，无需集成，但国内用户苦于无法安装中文字体。

本脚本由[**云浮else**](https://yfelse.notion.site/)编写，解决了国内用户无法安装中文字体的问题！

In [ ]:
import os
import sys
import platform
import subprocess
import requests
from pathlib import Path
import shutil

def get_system_font_dir():
    """获取系统字体目录"""
    system = platform.system()
    if system == "Windows":
        return "C:\\Windows\\Fonts"
    elif system == "Darwin":  # macOS
        return "/Library/Fonts"
    elif system == "Linux":
        # 对于Linux，我们使用用户字体目录而不是系统目录，避免权限问题
        return os.path.expanduser("~/.local/share/fonts")
    else:
        raise OSError(f"不支持的操作系统: {system}")

def install_font(font_url, font_name):
    """下载并安装字体"""
    font_dir = get_system_font_dir()
    
    # 创建字体目录（如果不存在）
    Path(font_dir).mkdir(parents=True, exist_ok=True)
    
    # 字体文件路径
    font_path = os.path.join(font_dir, font_name)
    
    # 检查字体是否已安装
    if os.path.exists(font_path):
        print(f"字体 {font_name} 已安装，跳过...")
        return True
    
    try:
        # 下载字体
        print(f"正在下载 {font_name}...")
        response = requests.get(font_url, stream=True)
        response.raise_for_status()
        
        # 保存字体文件
        with open(font_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        print(f"字体 {font_name} 安装成功！")
        
        # Linux系统需要更新字体缓存
        if platform.system() == "Linux":
            subprocess.run(["fc-cache", "-f", "-v"], check=True)
            print("字体缓存已更新")
            
        return True
        
    except Exception as e:
        print(f"安装 {font_name} 失败: {str(e)}")
        # 清理可能不完整的文件
        if os.path.exists(font_path):
            os.remove(font_path)
        return False

def install_matplotlib_font_config():
    """配置Matplotlib使用中文字体"""
    try:
        import matplotlib
        import matplotlib.pyplot as plt
        
        # 配置matplotlibrc文件
        matplotlib.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC", "Arial Unicode MS"]
        matplotlib.rcParams["axes.unicode_minus"] = False  # 解决负号显示问题
        
        # 保存配置
        plt.rcParams.update(matplotlib.rcParams)
        
        print("Matplotlib 字体配置已更新")
        return True
    except ImportError:
        print("未安装Matplotlib，跳过Matplotlib字体配置")
        return False
    except Exception as e:
        print(f"配置Matplotlib字体失败: {str(e)}")
        return False

def main():
    print(f"检测到操作系统: {platform.system()}")
    print(f"系统字体目录: {get_system_font_dir()}")
    
    # 中文字体列表 (名称, 下载链接)
    chinese_fonts = [
        ("SimHei.ttf", "https://github.com/StellarCN/scp_zh/raw/master/fonts/SimHei.ttf"),
        ("WenQuanYi Micro Hei.ttc", "https://github.com/StellarCN/scp_zh/raw/master/fonts/WenQuanYi%20Micro%20Hei.ttc"),
        ("NotoSansCJK-Regular.ttc", "https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTC/NotoSansCJK-Regular.ttc")
    ]
    
    # 安装字体
    print("\n开始安装中文字体...")
    for font_name, font_url in chinese_fonts:
        install_font(font_url, font_name)
    
    # 配置Matplotlib
    print("\n配置Matplotlib字体...")
    install_matplotlib_font_config()
    
    print("\n字体安装完成！请重启Jupyter Notebook以使字体生效。")
    print("如果仍然有中文显示问题，可以尝试清除浏览器缓存或重新启动计算机。")

if __name__ == "__main__":
    main()

如果你成功执行了该单元格，那么你已经安装好了中文字体！

## 你的第一个场景

Manim 通过渲染*场景（Scenes）* 来生成视频。这些是特殊的类，具有一个
`construct` 方法，描述应该渲染的动画。（对于本教程，如果你不太熟悉
Python 或面向对象编程术语，如*类*或*方法*，也没关系——但如果你想继续学习
Manim，应该考虑学习一个 Python 教程。）

说了这么多专业术语，让我们来看一个例子。运行下面的单元格来渲染并显示一个视频。

In [ ]:
%%manim -qm CircleToSquare

class CircleToSquare(Scene):
    def construct(self):
        blue_circle = Circle(color=BLUE, fill_opacity=0.5)
        green_square = Square(color=GREEN, fill_opacity=0.8)
        self.play(Create(blue_circle))
        self.wait()
        
        self.play(Transform(blue_circle, green_square))
        self.wait()

虽然这个例子的部分内容可能看起来不言自明，但我们还是会一步一步地讲解。首先，

    %%manim -qm CircleToSquare

是一个*魔术命令*，它只在 Jupyter 笔记本中有效。这与你从终端调用 `manim`
非常相似：标志 `-qm` 控制渲染质量，它是 `--quality=m`
的缩写，即中等渲染质量。这意味着视频将以 720p、30
帧/秒的速度渲染。（尝试将其改为 `-qh` 或 `-ql`
以获得*高*质量和*低*质量！）

最后，`CircleToSquare`
是你想要在这个特定单元格中渲染的场景类的名称，这也让我们看到了接下来的几行：

``` py
class CircleToSquare(Scene):
    def construct(self):
        [...]
```

这定义了一个名为 `CircleToSquare` 的 Manim 场景，并定义了一个自定义的
`construct` 方法，该方法充当视频的*蓝图*。`construct`
方法的内容描述了视频中具体要渲染的内容。

``` py
blue_circle = Circle(color=BLUE, fill_opacity=0.5)
green_square = Square(color=GREEN, fill_opacity=0.8)
```

前两行创建了一个 `Circle` 和一个 `Square`
对象，并指定了颜色和填充不透明度。然而，这些还没有被添加到场景中！要添加它们，你要么必须使用
`self.add`，要么…

``` py
self.play(Create(blue_circle))
self.wait()
```

…通过播放一个将 Manim
对象（*Mobject*）添加到场景中的动画。在该方法中，`self`
引用当前场景，`self.play(my_animation)`
可以理解为“*这个场景应该播放我的动画*”。

`Create` 就是这样一种动画，但还有许多其他动画（例如 `FadeIn` 或
`DrawBorderThenFill`——在上面试试它们！）。`self.wait()`
调用的作用正如你所期望的：它会暂停视频一段时间（默认情况下：一秒钟）。将其改为
`self.wait(2)` 可获得两秒钟的暂停，依此类推。

最后两行，

    self.play(Transform(blue_circle, green_square))
    self.wait()

负责从蓝色圆到绿色正方形的实际变换（之后还有一秒钟的暂停）。

## 定位和移动 Mobjects

新的问题：我们想要创建一个场景，其中在创建一个圆的同时，在其下方写一些文本。我们可以重用上面的蓝色圆，然后添加一些新代码：

In [ ]:
%%manim -qm HelloCircle

class HelloCircle(Scene):
    def construct(self):
        # blue_circle = Circle(color=BLUE, fill_opacity=0.5)
        # We can also create a "plain" circle and add the desired attributes via set methods:
        circle = Circle()
        blue_circle = circle.set_color(BLUE).set_opacity(0.5)
        
        label = Text("A wild circle appears!")
        label.next_to(blue_circle, DOWN, buff=0.5)
        
        self.play(Create(blue_circle), Write(label))
        self.wait()

显然，可以使用 `Text` Mobject 来渲染文本——而通过以下代码行可以实现所需的位置：
```py
label.next_to(blue_circle, DOWN, buff=0.5)
```
Mobjects 有一些用于定位的方法，`next_to` 是其中之一（`shift`、`to_edge`、`to_corner`、`move_to` 是另外一些——通过左侧的搜索栏在我们的[文档](https://docs.manim.community/)中查看它们！）。对于 `next_to`，传递的第一个参数（`blue_circle`）描述了我们的 `label` 应该放置在哪个对象旁边。第二个参数 `DOWN` 描述了方向（尝试改为 `LEFT`、`UP` 或 `RIGHT` 看看效果！）。最后，`buff=0.5` 控制 `blue_circle` 和 `label` 之间的“缓冲距离”，增大这个值会将 `label` 推得更靠下。

但还要注意，`self.play` 调用已经改变：可以向 `self.play` 传递多个动画参数，它们将同时播放。如果你想让它们一个接一个地播放，将 `self.play` 调用替换为以下行：
```py
self.play(Create(blue_circle))
self.play(Write(label))
```
看看会发生什么。

顺便说一下，Mobjects 自然也有与定位无关的方法：例如，要得到我们的蓝色圆，我们也可以创建一个默认的圆，然后设置颜色和不透明度：
```py
circle = Circle()
blue_transparent_circle = circle.set_color(BLUE)
blue_circle = blue_transparent_circle.set_opacity(0.5)
```
更简短的版本是：
```py
blue_circle = Circle().set_color(BLUE).set_opacity(0.5)
```
现在，我们将继续在对 `Circle` 的调用中直接设置属性。

## 动画方法调用：`.animate` 语法

在最后一个例子中，我们遇到了 `.next_to` 方法，这是许多（！）修改 Mobjects 的方法之一。但是，如果我们想在应用这些方法（例如 `shift` 移动某物、`rotate` 旋转一个 Mobject 或 `scale` 缩放它）时，对 Mobject 的变化进行动画处理，该怎么办呢？`.animate` 语法就是解决这个问题的方法，让我们来看一个例子。

In [ ]:
%%manim -qm CircleAnnouncement

class CircleAnnouncement(Scene):
    def construct(self):
        blue_circle = Circle(color=BLUE, fill_opacity=0.5)
        announcement = Text("Let us draw a circle.")
        
        self.play(Write(announcement))
        self.wait()
        
        self.play(announcement.animate.next_to(blue_circle, UP, buff=0.5))
        self.play(Create(blue_circle))

通常我们会使用 `announcement.next_to(blue_circle, UP, buff=0.5)` 来定位文本而不产生动画，而我们可以在方法调用前加上 `.animate`，将方法的应用转换为一个动画，然后可以使用 `self.play` 来播放这个动画。这适用于所有以某种方式修改 Mobject 的方法：

In [ ]:
%%manim -qm AnimateSyntax

class AnimateSyntax(Scene):
    def construct(self):
        triangle = Triangle(color=RED, fill_opacity=1)
        self.play(DrawBorderThenFill(triangle))
        self.play(triangle.animate.shift(LEFT))
        self.play(triangle.animate.shift(RIGHT).scale(2))
        self.play(triangle.animate.rotate(PI/3))

在第一个 play 调用中，三角形被创建；在第二个调用中，它被向左移动；然后在第三个调用中，它被向右移回并同时放大 2 倍；最后在第四个调用中，它被旋转了 $\pi/3$ 角度。修改一些值后再次运行上面的单元格，或者尝试其他方法，例如 `set_color`。

仔细观察上面场景中的最后一个动画（旋转），你可能会注意到这*实际上*并不是旋转。三角形被变换为自身的旋转版本，但在动画过程中，三角形的顶点并不是沿着弧线移动（就像三角形绕其中心旋转时那样），而是沿着直线移动，这使得动画给人的印象是三角形先稍微缩小，然后再放大。

这实际上**不是一个 bug**，而是 `.animate` 语法工作方式的结果：动画是通过指定起始状态（在示例中是 `triangle` Mobject）和最终状态（旋转后的 mobject，`triangle.rotate(PI/3)`）来构建的。然后 Manim 尝试在这两者之间进行插值，但实际上并不知道你希望平滑地旋转三角形。下面的例子清楚地说明了这一点：

In [ ]:
%%manim -qm DifferentRotations

class DifferentRotations(Scene):
    def construct(self):
        left_square = Square(color=BLUE, fill_opacity=0.7).shift(2*LEFT)
        right_square = Square(color=GREEN, fill_opacity=0.7).shift(2*RIGHT)
        self.play(left_square.animate.rotate(PI), Rotate(right_square, angle=PI), run_time=2)
        self.wait()

## 排版数学公式

Manim 支持渲染和动画化 LaTeX，这是一种常用于排版数学公式的标记语言。在[这个 30 分钟的教程](https://www.overleaf.com/learn/latex/Learn_LaTeX_in_30_minutes)中了解更多信息。

以下是在 Manim 中使用 LaTeX 的一个简单示例：

In [ ]:
%%manim -qm CauchyIntegralFormula

class CauchyIntegralFormula(Scene):
    def construct(self):
        formula = MathTex(r"[z^n]f(z) = \frac{1}{2\pi i}\oint_{\gamma} \frac{f(z)}{z^{n+1}}~dz")
        self.play(Write(formula), run_time=3)
        self.wait()

如这个例子所示，`MathTex` 允许渲染简单的（数学模式的）LaTeX 字符串。如果你想渲染“正常模式”的 LaTeX，请使用 `Tex` 代替。

当然，Manim 也可以帮助你可视化排版公式的变换。考虑以下示例：

In [ ]:
%%manim -qm TransformEquation

class TransformEquation(Scene):
    def construct(self):
        eq1 = MathTex("42 {{ a^2 }} + {{ b^2 }} = {{ c^2 }}")
        eq2 = MathTex("42 {{ a^2 }} = {{ c^2 }} - {{ b^2 }}")
        eq3 = MathTex(r"a^2 = \frac{c^2 - b^2}{42}")
        self.add(eq1)
        self.wait()
        self.play(TransformMatchingTex(eq1, eq2))
        self.wait()
        self.play(TransformMatchingShapes(eq2, eq3))
        self.wait()

在最后这个例子中，`eq1` 和 `eq2` 有一些双大括号的位置，而在普通的 LaTeX 中通常不会有。这是特殊的 Manim 符号，它以特定的方式对生成的 `Tex` Mobjects `eq1` 和 `eq2` 进行分组。

这种特殊符号在使用 `TransformMatchingTex` 动画时很有帮助：它会将具有相同 TeX 字符串的部分（例如，`a^2` 到 `a^2`）相互变换——如果没有这种特殊符号，整个方程会被视为一个长的 TeX 字符串。相比之下，`TransformMatchingShapes` 没那么智能：它只是尝试将“看起来相同”的形状相互变换——尽管如此，它仍然经常非常有用。

如果你已经学到了这里，你应该对这个库的基本用法有了一个初步的印象。下面你可以找到一些更高级的例子，它们说明了库中的一些更专门的概念。来吧，试着像修改上面的例子一样修改它们！浏览我们的[文档](https://docs.manim.community)，了解已经实现的功能——如果你想构建一些更复杂的对象，也可以查看源代码。

[社区](https://www.manim.community/discord/)当然也很乐意回答问题——我们希望你能与我们分享你很棒的项目！** 愉快地“manimating”吧！**

## 一些特定的例子

在深入研究这些例子之前请注意：它们展示的是专门的概念，旨在让你感受更复杂的场景是如何设置和编码的。这些例子没有附加说明，它们**不打算作为（入门级）学习资源**。

In [ ]:
%%manim -qm FormulaEmphasis

class FormulaEmphasis(Scene):
    def construct(self):
        product_formula = MathTex(
            r"\frac{d}{dx} f(x)g(x) =",
            r"f(x) \frac{d}{dx} g(x)",
            r"+",
            r"g(x) \frac{d}{dx} f(x)"
        )
        self.play(Write(product_formula))
        box1 = SurroundingRectangle(product_formula[1], buff=0.1)
        box2 = SurroundingRectangle(product_formula[3], buff=0.1)
        self.play(Create(box1))
        self.wait()
        self.play(Transform(box1, box2))
        self.wait()

In [ ]:
%%manim -qm PlotExample

class PlotExample(Scene):
    def construct(self):
        plot_axes = Axes(
            x_range=[0, 1, 0.05],
            y_range=[0, 1, 0.05],
            x_length=9,
            y_length=5.5,
            axis_config={
                "numbers_to_include": np.arange(0, 1 + 0.1, 0.1),
                "font_size": 24,
            },
            tips=False,
        )

        y_label = plot_axes.get_y_axis_label("y", edge=LEFT, direction=LEFT, buff=0.4)
        x_label = plot_axes.get_x_axis_label("x")
        plot_labels = VGroup(x_label, y_label)

        plots = VGroup()
        for n in np.arange(1, 20 + 0.5, 0.5):
            plots += plot_axes.plot(lambda x: x**n, color=WHITE)
            plots += plot_axes.plot(
                lambda x: x**(1 / n), color=WHITE, use_smoothing=False
            )

        extras = VGroup()
        extras += plot_axes.get_horizontal_line(plot_axes.c2p(1, 1, 0), color=BLUE)
        extras += plot_axes.get_vertical_line(plot_axes.c2p(1, 1, 0), color=BLUE)
        extras += Dot(point=plot_axes.c2p(1, 1, 0), color=YELLOW)
        title = Title(
            r"Graphs of $y=x^{\frac{1}{n}}$ and $y=x^n (n=1, 1.5, 2, 2.5, 3, \dots, 20)$",
            include_underline=False,
            font_size=40,
        )
        
        self.play(Write(title))
        self.play(Create(plot_axes), Create(plot_labels), Create(extras))
        self.play(AnimationGroup(*[Create(plot) for plot in plots], lag_ratio=0.05))

In [ ]:
%%manim -qm ErdosRenyiGraph

import networkx as nx

nxgraph = nx.erdos_renyi_graph(14, 0.5)

class ErdosRenyiGraph(Scene):
    def construct(self):
        G = Graph.from_networkx(nxgraph, layout="spring", layout_scale=3.5)
        self.play(Create(G))
        self.play(*[G[v].animate.move_to(5*RIGHT*np.cos(ind/7 * PI) +
                                         3*UP*np.sin(ind/7 * PI))
                    for ind, v in enumerate(G.vertices)])
        self.play(Uncreate(G))

In [ ]:
%%manim -qm CodeFromString

class CodeFromString(Scene):
    def construct(self):
        code = '''from manim import Scene, Square

class FadeInSquare(Scene):
    def construct(self):
        s = Square()
        self.play(FadeIn(s))
        self.play(s.animate.scale(2))
        self.wait()
'''
        rendered_code = Code(
            code_string=code, tab_width=4, background="window",
            language="python", paragraph_config=dict(font="Monospace")
        )
        self.play(Write(rendered_code))
        self.wait(2)

In [ ]:
%%manim -qm OpeningManim

class OpeningManim(Scene):
    def construct(self):
        title = Tex(r"This is some \LaTeX")
        basel = MathTex(r"\sum_{n=1}^\infty \frac{1}{n^2} = \frac{\pi^2}{6}")
        VGroup(title, basel).arrange(DOWN)
        self.play(
            Write(title),
            FadeIn(basel, shift=UP),
        )
        self.wait()

        transform_title = Tex("That was a transform")
        transform_title.to_corner(UP + LEFT)
        self.play(
            Transform(title, transform_title),
            LaggedStart(*[FadeOut(obj, shift=DOWN) for obj in basel]),
        )
        self.wait()

        grid = NumberPlane(x_range=(-10, 10, 1), y_range=(-6.0, 6.0, 1))
        grid_title = Tex("This is a grid")
        grid_title.scale(1.5)
        grid_title.move_to(transform_title)

        self.add(grid, grid_title)
        self.play(
            FadeOut(title),
            FadeIn(grid_title, shift=DOWN),
            Create(grid, run_time=3, lag_ratio=0.1),
        )
        self.wait()

        grid_transform_title = Tex(
            r"That was a non-linear function \\ applied to the grid"
        )
        grid_transform_title.move_to(grid_title, UL)
        grid.prepare_for_nonlinear_transform()
        self.play(
            grid.animate.apply_function(
                lambda p: p + np.array([np.sin(p[1]), np.sin(p[0]), 0])
            ),
            run_time=3,
        )
        self.wait()
        self.play(Transform(grid_title, grid_transform_title))
        self.wait()